# Trabalho prático II - Classificação

O objetivo deste trabalho é praticar os conceitos de aprendizado supervisionado que vimos em sala.

A sua tarefa será treinar um classificador para um conjunto de dados misterioso (se eu falasses qual é o objetivo do modelo, você encontraria soluções na Internet).

Baixe o arquivo [t2.tar.gz](https://drive.google.com/file/d/125plHKUzFGxHjjCiVJcTQG2bPG_zgDNV/view?usp=sharing). Descoprima este arquivo para encontrar outros quatro. Os arquivos `train_X.csv` e `train_y.csv` possuem os dados que você deve usar para treinar o modelo. O arquivo `test_X.csv` possui os objetos para os quais você deve encontrar as classes (testar o modelo). Por fim, o arquivo `test_example_y.csv` é um exemplo de como sua solução final deve ser organizada.

Cada linha dos arquivos `train_X.csv` e `test_X.csv` tem 15 campos descrevendo um objeto misterioso. O campo `id` representa o identificador do objeto, sendo que este campo não deve ser considerado em seu modelo. Os atributos a serem usados no modelo são os 14 campos rotulados de de `a` até `n`. Desses atributos:
- `b`, `d`, `f`, `g`, `h`, `i`, `j` e `n` são categóricos; e
- `a`, `c`, `e`, `k`, `l` e `m` são numéricos.

Cada linha do arquivo `train_y.csv` possui dois campos. O primeiro é o identificador de um objeto do arquivo `train_X.csv` e o segundo é a classe do respectivo objeto (0 ou 1).

Seu objetivo é encontrar as classes dos objetos do arquivo `test_X.csv` e mostrar como chegou em sua solução! Os dados do arquivo de teste foram obtido a partir de uma amostra aleatório do todo. Ou seja, um modelo bem treinado, e sem _overfitting_, em `train_X.csv` e `train_y.csv` se sairá bem em `test_X.csv`.

**Data de entrega:** dia 4 de julho de 2018.

**Grupo:** de até 3 pessoas, mas duas pessoas do mesmo grupo no trabalho 1 não podem pertencer ao mesmo grupo nesse trabalho.

**Valor:** 20% da nota do semestre.

Os três seguintes pontos descrevem o que obrigatoriamente deve ser entregue, com seu respectivo valor.

1 - **[10 pontos]** Este notebook com todo seu código e resultados (números, tabelas e gráficos). Você pode usar qualquer um dos métodos que estudamos ou alguma de suas variações próximas. Se estiver na dúvida se pode usar um método, basta perguntas no Piazza. Comentários e justificativas no notebook não serão considerados para sua nota.
O notebook deve ser enviado para o email do professor.

2 - **[8 pontos]** Um relatório digitado contendo: capa, introdução, metodologia, resultados, conclusão e referências. O relatório deve ter no máximo 10 páginas, com coluna simples, fonte 11, espaçamento 1.5 e margens de 2cm. A seção de metodologia deve conter uma descrição detalhada dos passos seguidos (não incluir código no relatório). A seção de resultados deve conter obrigatoriamente: uma caracterização descritiva dos dados, matriz de confusão das predições, _precision_, _recall_, _F1 score_ e acurácia. Todas as métricas de predição devem ser calculadas a partir dos arquivos de treinamento por meio de validação cruzada.
O relatório deve ser enviado para o email do professor.

3 - **[2 pontos + equivalente a lista extra pela classificação]** A sua predição final do arquivo `test_X.csv` deve ser enviada para o professor por email. O formato deve ser o mesmo do arquivo `train_y.csv`, assim como exemplificado em `test_example_y.csv` (mas repare que as classes desse último arquivo foram gerados de forma aleatória). Em outras palavras, o arquivo a ser entregue deve ter dois campos. O campo `id` é o identificador do objeto em `test_X.csv` e o campo `label` é a classe que seu modelo encontrou para o objeto em questão. A primeira linha do arquivo deve conter os nomes das colunas.
A entrega desse arquivo é obrigatória e vale dois pontos. Além disso, o trabalho com maior _F1 score_ ganhará o equivalente a 100% de uma lista extra. O trabalho com o pior _F1 score_ não ganhará nota extra alguma. Os demais trabalhos terão nota proporcional.
O professor se reserva o direito de anular esse quesito (nota extra) se houver indícios de má conduta durante a competição.

**Kaggle:** Estou tentando criar uma competição para esse trabalho na plataforma _Kaggle_. Se eu conseguir, compartilho o _link_ com você no _Piazza_.

In [50]:
# your code starts here...
# Codigo feito em python 3.6
#Lúcio Paulo Reis – 78379
#Rafael Victor Costa Braz – 85262
#Igor Oliveira – 77431
#Tulio Mendonça Cunha dos Santos – 77428

import pandas as pd
from pandas import DataFrame as df
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_validate, cross_val_predict
from sklearn.metrics import confusion_matrix 
from scipy.stats import randint as sp_randint
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [59]:
df_x = pd.read_csv('train_X.csv', index_col='id')
sr_y = pd.read_csv('train_y.csv', index_col='id', squeeze=True)#Load the values as a Serie
df_test_x = pd.read_csv('test_X.csv', index_col='id')
categories=['b','d','f','g','h','i','j','n']

In [52]:
def show_scores(model, x, y, cv=5):
    scoring = ['f1', 'recall', 'accuracy', 'precision']
    scores = cross_validate(model, x, y, cv=cv, scoring=scoring, n_jobs=-1)
    print("Test Precision: %f" 
          % scores['test_precision'].mean())
    print("Train Precision: %f" 
          % scores['train_precision'].mean())
    print("Test Accuracy: %f" 
          % scores['test_accuracy'].mean())
    print("Train Accuracy: %f" 
          % scores['train_accuracy'].mean())
    print("Test Recall: %f" 
          % scores['test_recall'].mean())
    print("Train Recall: %f" 
          % scores['train_recall'].mean())
    print("Test f1: %f" 
          % scores['test_f1'].mean())
    print("Train f1: %f" 
          % scores['train_f1'].mean())
    return scores

def predict(model, x, y=None, cv=5):
    y_pred = cross_val_predict(model,x, y,cv=cv)
    conf_mat = pd.DataFrame(confusion_matrix(y,y_pred), 
                            columns=["Predito como 0", "Predito como 1"], 
                            index=['True 0', 'True 1'])
    print(conf_mat)
    return y_pred

In [56]:
rnd_clf = RandomForestClassifier(n_jobs=-1, 
                                 bootstrap=True,
                                 criterion='gini',
                                 max_depth=None,
                                 max_features=8,
                                 min_samples_leaf=1,
                                 max_leaf_nodes=23,
                                 min_samples_split=9,
                                 class_weight="balanced",
                                 n_estimators=29)

ada_clf = AdaBoostClassifier(rnd_clf,
                              algorithm="SAMME.R",
                              learning_rate=0.75,
                              n_estimators=20)

In [61]:
show_scores(ada_clf, df_x, sr_y);
predict(ada_clf, df_x, sr_y);

Test Precision: 0.618913
Train Precision: 0.637710
Test Accuracy: 0.834021
Train Accuracy: 0.847478
Test Recall: 0.860823
Train Recall: 0.891156
Test f1: 0.720053
Train f1: 0.743423
        Predito como 0  Predito como 1
True 0           24940            5309
True 1            1385            8588


array([0, 0, 0, ..., 0, 1, 1], dtype=int64)

In [62]:
ada_clf.fit(df_x, sr_y)
class_pred = ada_clf.predict(df_test_x)

In [65]:
pd.DataFrame([i for i in zip(df_test_x.index, class_pred)], columns=['id','label']).to_csv("test_y.csv", index=False);